In [ ]:
from collections import Counter
import string, shutil, pprint, nltk

In [ ]:
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("averaged_perceptron_tagger")
nltk.download("stopwords")
nltk.download('vader_lexicon')

In [ ]:
CHAT_TEXT_FILEPATH_LIST = ["../../data/zoom_meeting_saved_chat_1.txt", "../../data/zoom_meeting_saved_chat_2.txt"]
COMBINED_TEXT_FILEPATH = "../../data/zoom_meeting_saved_combined.txt"

## Chat Data Dictionary
Build data dictionary from text file to match response to username

In [ ]:
def remove_ascii(string):
    updated_string = "".join(i for i in string if ord(i) < 128)
    updated_string = updated_string.replace("\n", " ")
    updated_string  = updated_string.replace("\r", " ")
    
    return updated_string

In [ ]:
def merge_text_files(filepath, list_of_files):
    merged_files = []
    
    with open(filepath, "w") as outfile:
    
        for file_name in list_of_files:
        
            with open(file_name) as infile:
                merged_files.append(shutil.copyfileobj(infile, outfile))
            
        return merged_files

In [ ]:
merged_text_files = merge_text_files(COMBINED_TEXT_FILEPATH, CHAT_TEXT_FILEPATH_LIST)

In [ ]:
def read_text_file(filepath):
    chat_text_file = open(filepath, "r")
    lines = chat_text_file.readlines()
    messages = []

    for line in lines:
        line = line.lower()
        line = line.encode("ascii", errors="ignore")
        line = line.decode()
        split_lines = line.split("\t")
        
        if len(split_lines) > 2:
            join_message = " ".join(split_lines[1:])
            messages.append([join_message])

        if len(split_lines) <= 2:
            messages.append(split_lines[1:])
            
    return messages

In [ ]:
chat_messages_list = read_text_file(COMBINED_TEXT_FILEPATH)

In [ ]:
def get_usernames(messages_list):
    zoom_usernames = []
    
    for message in messages_list:
        
        for text in message:
            split_colon = text.split(":")
            
            if split_colon[0] != "":
                zoom_usernames.append(split_colon[0].strip())
        
    return list(set(zoom_usernames))

In [ ]:
def filter_usernames(username_list, remove_names_list):
    filter_names = []
    
    for username in username_list:
        
        if username not in remove_names_list:
            filter_names.append(username)
    
    return filter_names

In [ ]:
user_name_list = get_usernames(chat_messages_list)
names_to_remove = ["", "", ""]
filtered_names = filter_usernames(user_name_list, names_to_remove)

In [ ]:
def create_chat_data_dictionary(message_list, filtered_name_list):
    name_dictionary = {}
    counter = 0
    
    for index in message_list:

        for username in index:
            split_colon = username.split(":")

            if split_colon[0] in filtered_name_list:
                counter+=1
                formatted_name = "{}_{}".format(split_colon[0].strip(), counter)
                name_dictionary[formatted_name]= split_colon[1].strip()

    return name_dictionary

In [ ]:
chat_data_dictionary = create_chat_data_dictionary(chat_messages_list, filtered_names)
pp.pprint(chat_data_dictionary)